# Typing Iterables

In [14]:
from typing import (
    Deque, Dict, FrozenSet, List, Optional, Sequence, Set, Tuple, Union
)

from pydantic import BaseModel


class Model(BaseModel):
    simple_list: list = None
    list_of_ints: List[int] = None

    simple_tuple: tuple = None
    tuple_of_different_types: Tuple[int, float, str, bool] = None

    simple_dict: dict = None
    dict_str_float: Dict[str, float] = None

    simple_set: set = None
    set_bytes: Set[bytes] = None
    # FrozenSet 为不可变集合，一旦创建了便不可更改，没有 `add` 和 `remove` 方法
    frozen_set: FrozenSet[int] = None

    str_or_bytes: Union[str, bytes] = None
    none_or_str: Optional[str] = None

    sequence_of_ints: Sequence[int] = None

    compound: Dict[Union[str, bytes], List[Set[int]]] = None

    deque: Deque[int] = None

model = Model(
    simple_list=['1', '2', '3'],
    list_of_ints=[1, 2, 3],
    simple_tuple=(1, 2, 3, 4),
    tuple_of_different_types=(4, 3.0, '2', True),
    simple_dict={'a': 1, b'b': 2},
    dict_str_float={'a': 1, b'b': 2},
    simple_set=('a', 'b', 'c'),
    set_bytes=('a', 'b', 'c'),
    frozen_set=(1, 2, 3),
    str_or_bytes=b'abc',  # 也可以直接使用 abc
    none_or_str=None,
    sequence_of_ints=[1, 2, 3, 4],  # 也可以使用 (1, 2, 3, 4)
    compound={"abc": [(1, 2, 3), ['1', '4']]},
    deque=[1, 2, 3],
)


print("👻 Builtin list: ", model.simple_list)
print("👻 Typing List[int]: ", model.list_of_ints)

print("👻 Builtin tuple: ", model.simple_tuple)
print("👻 Typing Tuple[int, float, str, bool]: ", model.tuple_of_different_types)

print("👻 Builtin dict: ", model.simple_dict)
print("👻 Typing Dict[str, float]: ", model.dict_str_float)

print("👻 Builtin set: ", model.simple_set)
print("👻 Typing Set[bytes]: ", model.set_bytes)
print("👻 Typing FrozenSet[int]: ", model.frozen_set)

print("👻 Typing Union[str, bytes]: ", model.str_or_bytes)
print("👻 Typing Optional[str]: ", model.none_or_str)
print("👻 Typing Sequence[int]: ", model.sequence_of_ints)

print("👻 Typing Sequence[int]: ", model.sequence_of_ints)
print("👻 Typing Dict[Union[str, bytes], List[Set[int]]]: ", model.compound)
print("👻 Typing Deque[int]", model.deque)

👻 Builtin list:  ['1', '2', '3']
👻 Typing List[int]:  [1, 2, 3]
👻 Builtin tuple:  (1, 2, 3, 4)
👻 Typing Tuple[int, float, str, bool]:  (4, 3.0, '2', True)
👻 Builtin dict:  {'a': 1, b'b': 2}
👻 Typing Dict[str, float]:  {'a': 1.0, 'b': 2.0}
👻 Builtin set:  {'c', 'b', 'a'}
👻 Typing Set[bytes]:  {b'c', b'b', b'a'}
👻 Typing FrozenSet[int]:  frozenset({1, 2, 3})
👻 Typing Union[str, bytes]:  abc
👻 Typing Optional[str]:  None
👻 Typing Sequence[int]:  [1, 2, 3, 4]
👻 Typing Sequence[int]:  [1, 2, 3, 4]
👻 Typing Dict[Union[str, bytes], List[Set[int]]]:  {'abc': [{1, 2, 3}, {1, 4}]}
👻 Typing Deque[int] deque([1, 2, 3])


# Infinite Generators

In [16]:
from typing import Iterable
from pydantic import BaseModel


class Model(BaseModel):
    infinite: Iterable[int]


def infinite_ints():
    i = 0
    while True:
        yield i
        i += 1


m = Model(infinite=infinite_ints())
m

Model(infinite=<generator object infinite_ints at 0x000001EC9D31BDD0>)

In [17]:
for i in m.infinite:
    print(i)

    if i == 3:
        break

0
1
2
3


## Validating the first value

In [18]:
import itertools
from typing import Iterable
from pydantic import BaseModel, validator, ValidationError
from pydantic.fields import ModelField


class Model(BaseModel):
    infinite: Iterable[int]

    @validator('infinite')
    # You don't need to add the "ModelField", but it will help your
    # editor give you completion and catch errors
    def infinite_first_int(cls, iterable, field: ModelField):
        first_value = next(iterable)

        if field.sub_fields:
            # The Iterable had a parameter type, in this case it's int
            # We use it to validate the first value
            sub_field = field.sub_fields[0]
            v, error = sub_field.validate(first_value, {}, loc='first_value')

            if error:
                raise ValidationError([error], cls)

        # This creates a new generator that returns the first value and then
        # the rest of the values from the (already started) iterable
        return itertools.chain([first_value], iterable)


def infinite_ints():
    i = 0
    while True:
        yield i
        i += 1


m = Model(infinite=infinite_ints())
m

Model(infinite=<itertools.chain object at 0x000001EC991D7820>)

In [19]:
def infinite_strs():
    while True:
        for letter in 'allthesingleladies':
            yield letter


try:
    Model(infinite=infinite_strs())
except ValidationError as e:
    print(e)

1 validation error for Model
infinite -> first_value
  value is not a valid integer (type=type_error.integer)


# Unions

In [20]:
from uuid import UUID
from typing import Union
from pydantic import BaseModel


class User(BaseModel):
    id: Union[int, str, UUID]
    name: str


user_01 = User(id=123, name='John Doe')
print(user_01)
print(user_01.id)

user_02 = User(id='1234', name='John Doe')
print(user_02)
print(user_02.id)

user_03_uuid = UUID('cf57432e-809e-4353-adbd-9d5c0d733868')
user_03 = User(id=user_03_uuid, name='John Doe')
print(user_03)
print(user_03.id)
print(user_03_uuid.int)

id=123 name='John Doe'
123
id=1234 name='John Doe'
1234
id=275603287559914445491632874575877060712 name='John Doe'
275603287559914445491632874575877060712
275603287559914445491632874575877060712


In [21]:
from uuid import UUID
from typing import Union
from pydantic import BaseModel


class User(BaseModel):
    id: Union[UUID, int, str]
    name: str


user_03_uuid = UUID('cf57432e-809e-4353-adbd-9d5c0d733868')
user_03 = User(id=user_03_uuid, name='John Doe')
print(user_03)
print(user_03.id)
print(user_03_uuid.int)

id=UUID('cf57432e-809e-4353-adbd-9d5c0d733868') name='John Doe'
cf57432e-809e-4353-adbd-9d5c0d733868
275603287559914445491632874575877060712


## Discriminated Unions

In [22]:
from typing import Literal, Union
from pydantic import BaseModel, Field, ValidationError


class Cat(BaseModel):
    pet_type: Literal['cat']
    meows: int


class Dog(BaseModel):
    pet_type: Literal['dog']
    barks: float


class Lizard(BaseModel):
    pet_type: Literal['reptile', 'lizard']
    scales: bool


class Model(BaseModel):
    pet: Union[Cat, Dog, Lizard] = Field(..., discriminator='pet_type')
    n: int


Model(pet={'pet_type': 'dog', 'barks': 3.14}, n=1)

Model(pet=Dog(pet_type='dog', barks=3.14), n=1)

In [23]:
try:
    Model(pet={'pet_type': 'dog'}, n=1)
except ValidationError as e:
    print(e)

1 validation error for Model
pet -> Dog -> barks
  field required (type=value_error.missing)


## Nested Discriminated Unions

In [24]:
from typing import Literal, Union
from typing_extensions import Annotated
from pydantic import BaseModel, Field, ValidationError


class BlackCat(BaseModel):
    pet_type: Literal['cat']
    color: Literal['black']
    black_name: str


class WhiteCat(BaseModel):
    pet_type: Literal['cat']
    color: Literal['white']
    white_name: str


Cat = Annotated[Union[BlackCat, WhiteCat], Field(discriminator='color')]


class Dog(BaseModel):
    pet_type: Literal['dog']
    name: str


Pet = Annotated[Union[Cat, Dog], Field(discriminator='pet_type')]


class Model(BaseModel):
    pet: Pet
    n: int


m = Model(pet={'pet_type': 'cat', 'color': 'black', 'black_name': 'felix'}, n=1)
m

Model(pet=BlackCat(pet_type='cat', color='black', black_name='felix'), n=1)

In [25]:
try:
    Model(pet={'pet_type': 'cat', 'color': 'red'}, n='1')
except ValidationError as e:
    print(e)

1 validation error for Model
pet -> Union[BlackCat, WhiteCat]
  No match for discriminator 'color' and value 'red' (allowed values: 'black', 'white') (type=value_error.discriminated_union.invalid_discriminator; discriminator_key=color; discriminator_value=red; allowed_values='black', 'white')
